In [11]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [12]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_non-reverse'
df = pd.read_pickle(f'{name_df}.pkl')

In [13]:
df

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29755,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
29756,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
29757,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
29758,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
len(df)

29760

In [15]:
new_columns = [f'Col_{i}' for i in range(1, 50 + 1)]
df_sort = df.sort_values(by = new_columns).reset_index(drop=True)

In [16]:
# Check the current data types of the DataFrame
print("Before Conversion:")
print(df_sort.dtypes)
print("\n")

for i in df_sort.columns:
    df_sort[i] = df_sort[i].astype(int)

# Check the data types after conversion
print("After Conversion:")
print(df_sort.dtypes)

Before Conversion:
Col_1     int32
Col_2     int32
Col_3     int32
Col_4     int32
Col_5     int32
Col_6     int32
Col_7     int32
Col_8     int32
Col_9     int32
Col_10    int32
Col_11    int32
Col_12    int32
Col_13    int32
Col_14    int32
Col_15    int32
Col_16    int32
Col_17    int32
Col_18    int32
Col_19    int32
Col_20    int32
Col_21    int32
Col_22    int32
Col_23    int32
Col_24    int32
Col_25    int32
Col_26    int32
Col_27    int32
Col_28    int32
Col_29    int32
Col_30    int32
Col_31    int32
Col_32    int32
Col_33    int32
Col_34    int32
Col_35    int32
Col_36    int32
Col_37    int32
Col_38    int32
Col_39    int32
Col_40    int32
Col_41    int32
Col_42    int32
Col_43    int32
Col_44    int32
Col_45    int32
Col_46    int32
Col_47    int32
Col_48    int32
Col_49    int32
Col_50    int32
dtype: object


After Conversion:
Col_1     int32
Col_2     int32
Col_3     int32
Col_4     int32
Col_5     int32
Col_6     int32
Col_7     int32
Col_8     int32
Col_9     int32
Col

In [17]:
len(df_sort.columns)

50

In [18]:
# Step 1: Prepare Data
amount_features = len(df_sort.columns) - SIZE*SIZE #the previous boards
features = df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  26784
len x test:  2976
len y train:  26784
len y test:  2976
639 12


,train,test
accuracy,1.0,1.0
precision,1.0,1.0
recall,1.0,1.0
f1,1.0,1.0


In [19]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\forward_model\\decision_tree_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.725956  0.737567
precision  0.601863  0.612551
recall     0.447989  0.474630
f1         0.513650  0.534842
63 5
              train      test
accuracy   0.734058  0.746976
precision  0.670305  0.694165
recall     0.347781  0.364693
f1         0.457956  0.478170
123 6
              train      test
accuracy   0.763217  0.775874
precision  0.679851  0.690834
recall     0.504623  0.533827
f1         0.579276  0.602266
227 7
              train      test
accuracy   0.794093  0.805780
precision  0.692430  0.699134
recall     0.652335  0.682875
f1         0.671785  0.690909
393 8
              train      test
accuracy   0.872760  0.861223
precision  0.803753  0.777894
recall     0.801896  0.788584
f1         0.802823  0.783202
597 9
              train      test
accuracy   0.992981  0.993280
precision  0.983215  0.987368
recall     0.995261  0.991543
f1         0.989202  0.989451
621 10
              train      test
accuracy   0.996154  0.997312

In [20]:
amount_features = len(df_sort.columns) - SIZE*SIZE #the previous boards
features = df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 20, 42)
    name_file = f"{PATH_MODELS}\\forward\\dt_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
1045 20
              train      test
accuracy   0.999963  0.992944
precision  1.000000  0.992577
recall     0.999884  0.985263
f1         0.999942  0.988906
1
1503 19
           train      test
accuracy     1.0  0.978831
precision    1.0  0.963580
recall       1.0  0.970650
f1           1.0  0.967102
2
947 17
           train      test
accuracy     1.0  0.995968
precision    1.0  0.991895
recall       1.0  0.995931
f1           1.0  0.993909
3
1013 20
              train      test
accuracy   0.999963  0.994288
precision  1.000000  0.987448
recall     0.999881  0.994731
f1         0.999941  0.991076
4
663 13
           train      test
accuracy     1.0  0.999328
precision    1.0  0.997890
recall       1.0  1.000000
f1           1.0  0.998944
5
617 11
           train  test
accuracy     1.0   1.0
precision    1.0   1.0
recall       1.0   1.0
f1           1.0   1.0
6
933 20
              train      test
accuracy   0.999925  0.992272
precision  1.000000  0.989418
recall     0.999769  0.9